In [1]:
#1 step installing requirements and repo
print('\033[1;32mInstalling requirements...')

from IPython.display import clear_output
!conda install -c conda-forge libglib -y & conda update -n base conda -y

!pip install tqdm

clear_output()
print('\033[1;32mDone! Conda installed!')

# Clone the Stable Diffusion Web UI repository
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd stable-diffusion-webui

# Uncomment this line if you need to install the requirements
%pip install -r requirements.txt --quiet

# ControlNet extension
#reactor extenshion
%cd extensions/
!git clone https://github.com/epic-miner/sd-webui-reactor.git
!git clone https://github.com/Mikubill/sd-webui-controlnet.git
%cd ..

clear_output()
print('\033[1;32mDone!')

Done!


In [ ]:
# Install pyngrok
!pip install pyngrok

import os
from pyngrok import ngrok
import subprocess
import threading

# Prompt user for ngrok authentication token
ngrok_auth_token = input("Enter your ngrok authentication token: ")
ngrok.set_auth_token(ngrok_auth_token)

# Function to start the Web UI
def start_webui(device="cpu"):
    os.chdir("/home/studio-lab-user/stable-diffusion-webui")
    if device == "cpu":
        subprocess.run(["python", "launch.py", "--port", "7860", "--skip-torch-cuda-test"])
    elif device == "gpu":
        subprocess.run(["python", "launch.py", "--port", "7860"])

# Function to start ngrok
def start_ngrok():
    public_url = ngrok.connect(7860)
    print(f'Starting ngrok tunnel: {public_url}')

# Prompt user for device selection
device_option = input("Select device (1 for CPU, 2 for GPU): ").strip()

# Start the Web UI in a separate thread based on device option
if device_option == "1":
    webui_thread = threading.Thread(target=start_webui, args=("cpu",))
    webui_thread.start()
    # Start ngrok for CPU
    start_ngrok()
elif device_option == "2":
    webui_thread = threading.Thread(target=start_webui, args=("gpu",))
    webui_thread.start()
    # Start ngrok for GPU
    start_ngrok()
else:
    print("Invalid option selected.")
    exit(1)

# Keep the script running to maintain the ngrok tunnel
webui_thread.join()


In [ ]:
#check storage

import shutil

total, used, free = shutil.disk_usage("/")

print("Total: %d GB" % (total // (2**30)))
print("Used: %d GB" % (used // (2**30)))
print("Free: %d GB" % (free // (2**30)))



In [ ]:
#chek avalable storage and delete file by selecting numbrs

import os
import shutil
import subprocess

def check_disk_usage():
    """Check the current disk usage."""
    total, used, free = shutil.disk_usage("/")
    return free // (2**30)  # Free space in GB

def get_largest_files_and_dirs(path="/", n=10):
    """Get the largest files and directories in the specified path."""
    result = subprocess.run(
        ["du", "-ah", path], stdout=subprocess.PIPE, text=True
    )
    lines = result.stdout.splitlines()
    sizes = []
    for line in lines:
        size, name = line.split(maxsplit=1)
        if size.endswith('G'):
            size = float(size[:-1]) * 1024 ** 3
        elif size.endswith('M'):
            size = float(size[:-1]) * 1024 ** 2
        elif size.endswith('K'):
            size = float(size[:-1]) * 1024
        else:
            size = float(size)
        sizes.append((size, name))
    sizes.sort(reverse=True, key=lambda x: x[0])
    return sizes[:n]

def delete_file(path):
    """Delete the specified file or directory."""
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

def main():
    print("Checking current disk usage...")
    free_space = check_disk_usage()
    if free_space > 5:
        print("You have enough free space.")
        return

    print("\nListing the largest files and directories...")
    largest_items = get_largest_files_and_dirs()
    for i, (size, name) in enumerate(largest_items, start=1):
        print(f"{i}. {size / (1024 ** 2):.2f} MB - {name}")

    while True:
        try:
            choice = int(input("\nEnter the number of the file/directory to delete (or 0 to exit): "))
            if choice == 0:
                break
            if choice < 1 or choice > len(largest_items):
                print("Invalid choice. Please enter a number from the list.")
                continue
            _, path = largest_items[choice - 1]
            delete_file(path)
            print(f"Deleted {path}.")
            free_space = check_disk_usage()
            print(f"Free space after deletion: {free_space} GB")
            if free_space > 5:
                print("You now have enough free space.")
                break
        except ValueError:
            print("Invalid input. Please enter a number.")

if __name__ == "__main__":
    main()
